In [49]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from ipynb.fs.full.UseCase4_PredictTravelInsuranceClaim_FE import *

In [3]:
file_path=r'C:\Users\ADMIN\Desktop\UseCase2_EDAOutput.xlsx'

In [4]:
customer_data=pd.read_excel(file_path)

In [5]:
#Splitiing into train and test data before doing encoding on categorical values.
X=customer_data[['vehicle_claim','umbrella_limit','capital-gains','collision_type','incident_severity','incident_state','incident_type','insured_occupation','authorities_contacted','auto_model','insured_hobbies','insured_zip_new','target']]
y=customer_data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                    random_state=0)
print(X_train.shape, X_test.shape)
print(y_train.shape,y_test.shape)

(700, 13) (300, 13)
(700,) (300,)


In [9]:
X.head()

,vehicle_claim,umbrella_limit,capital-gains,collision_type,incident_severity,incident_state,incident_type,insured_occupation,authorities_contacted,auto_model,insured_hobbies,insured_zip_new,target
0,52080,0,53300,Side Collision,Major Damage,SC,Single Vehicle Collision,craft-repair,Police,92x,sleeping,46,1
1,3510,5000000,0,Missing,Minor Damage,VA,Vehicle Theft,machine-op-inspct,Police,E400,reading,46,1
2,23100,5000000,35100,Rear Collision,Minor Damage,NY,Multi-vehicle Collision,sales,Police,RAM,board-games,43,0
3,50720,6000000,48900,Front Collision,Major Damage,OH,Single Vehicle Collision,armed-forces,Police,Tahoe,board-games,60,1
4,4550,6000000,66000,Missing,Minor Damage,NY,Vehicle Theft,sales,None,RSX,board-games,61,0


Now lets create categorical columns to numerical values.

In [10]:
#Trying to use monotonicity encoding technique
def CategoricalImputation_Monotonicity(X_train,X_test,variable,target_variable):
    ordered_labels=X_train.groupby([variable])[target_variable].mean().sort_values().index
    ordinal_label = {k:i for i, k in enumerate(ordered_labels, 1)}
    X_train[variable+'_ordered']=X_train[variable].map(ordinal_label)
    X_test[variable+'_ordered']=X_test[variable].map(ordinal_label)

In [11]:
#Trying to use monotonicity encoding for collision_type and incident_state
CategoricalImputation_Monotonicity(X_train,X_test,'collision_type','target')
CategoricalImputation_Monotonicity(X_train,X_test,'incident_type','target')
CategoricalImputation_Monotonicity(X_train,X_test,'incident_state','target')


In [12]:
X_train.head()

,vehicle_claim,umbrella_limit,capital-gains,collision_type,incident_severity,incident_state,incident_type,insured_occupation,authorities_contacted,auto_model,insured_hobbies,insured_zip_new,target,collision_type_ordered,incident_type_ordered,incident_state_ordered
105,3360,0,34300,Missing,Minor Damage,WV,Vehicle Theft,prof-specialty,None,E400,kayaking,45,0,1,1,1
68,49840,0,43300,Front Collision,Major Damage,WV,Single Vehicle Collision,machine-op-inspct,Ambulance,A5,movies,47,0,3,4,1
479,66570,0,43700,Front Collision,Major Damage,WV,Single Vehicle Collision,sales,Fire,ML350,yachting,46,1,3,4,1
399,57690,6000000,0,Rear Collision,Minor Damage,NY,Single Vehicle Collision,armed-forces,Ambulance,Malibu,exercise,61,0,4,4,3
434,46620,0,0,Rear Collision,Major Damage,NC,Single Vehicle Collision,priv-house-serv,Fire,CRV,skydiving,43,0,4,4,2


In [13]:
#Use ordinal encoding for incident_severity
incident_severity_map = {'Trivial Damage':1,
               'Minor Damage':2,
               'Major Damage':3,
               'Total Loss':4,
               
}

X_train['incident_severity_number'] = X_train.incident_severity.map(incident_severity_map)
X_test['incident_severity_number'] = X_test.incident_severity.map(incident_severity_map)
X_train.head()

,vehicle_claim,umbrella_limit,capital-gains,collision_type,incident_severity,incident_state,incident_type,insured_occupation,authorities_contacted,auto_model,insured_hobbies,insured_zip_new,target,collision_type_ordered,incident_type_ordered,incident_state_ordered,incident_severity_number
105,3360,0,34300,Missing,Minor Damage,WV,Vehicle Theft,prof-specialty,None,E400,kayaking,45,0,1,1,1,2
68,49840,0,43300,Front Collision,Major Damage,WV,Single Vehicle Collision,machine-op-inspct,Ambulance,A5,movies,47,0,3,4,1,3
479,66570,0,43700,Front Collision,Major Damage,WV,Single Vehicle Collision,sales,Fire,ML350,yachting,46,1,3,4,1,3
399,57690,6000000,0,Rear Collision,Minor Damage,NY,Single Vehicle Collision,armed-forces,Ambulance,Malibu,exercise,61,0,4,4,3,2
434,46620,0,0,Rear Collision,Major Damage,NC,Single Vehicle Collision,priv-house-serv,Fire,CRV,skydiving,43,0,4,4,2,3


In [17]:
#Using weight of evidence encoding technique
def CategoricalImputation_WOE(X_train,X_test,variable,target_variable):
    # now we calculate the probability of target=1 
    prob_df = X_train.groupby([variable])[target_variable].mean()
    prob_df = pd.DataFrame(prob_df)
    
    # and now the probability of target = 0 
    # and we add it to the dataframe
    prob_df['target_0'] = 1-prob_df[target_variable]
    prob_df.loc[prob_df[target_variable] == 0, target_variable] = 0.001
    prob_df['WoE'] = np.log(prob_df[target_variable]/prob_df['target_0'])
    ordered_labels = prob_df['WoE'].to_dict()
    X_train[variable+'_ordered'] = X_train[variable].map(ordered_labels)
    X_test[variable+'_ordered'] = X_test[variable].map(ordered_labels)

In [18]:
#Use WOE encoding for other categorical columns
CategoricalImputation_WOE(X_train,X_test,'insured_zip_new','target')
CategoricalImputation_WOE(X_train,X_test,'insured_hobbies','target')
CategoricalImputation_WOE(X_train,X_test,'insured_occupation','target')
CategoricalImputation_WOE(X_train,X_test,'authorities_contacted','target')
CategoricalImputation_WOE(X_train,X_test,'auto_model','target')
CategoricalImputation_WOE(X_train,X_test,'incident_severity','target')

In [19]:
X_train.head()

,vehicle_claim,umbrella_limit,capital-gains,collision_type,incident_severity,incident_state,incident_type,insured_occupation,authorities_contacted,auto_model,...,collision_type_ordered,incident_type_ordered,incident_state_ordered,incident_severity_number,insured_zip_new_ordered,insured_hobbies_ordered,insured_occupation_ordered,authorities_contacted_ordered,auto_model_ordered,incident_severity_ordered
105,3360,0,34300,Missing,Minor Damage,WV,Vehicle Theft,prof-specialty,None,E400,...,1,1,1,2,-0.847298,-2.538974,-1.142097,-2.772589,-1.029619,-2.239784
68,49840,0,43300,Front Collision,Major Damage,WV,Single Vehicle Collision,machine-op-inspct,Ambulance,A5,...,3,4,1,3,-0.584513,-1.145132,-1.526056,-1.021651,-0.559616,0.362905
479,66570,0,43700,Front Collision,Major Damage,WV,Single Vehicle Collision,sales,Fire,ML350,...,3,4,1,3,-1.145132,-0.847298,-1.049822,-0.871839,0.000000,0.362905
399,57690,6000000,0,Rear Collision,Minor Damage,NY,Single Vehicle Collision,armed-forces,Ambulance,Malibu,...,4,4,3,2,-1.399366,-1.440362,-1.315677,-1.021651,-1.845827,-2.239784
434,46620,0,0,Rear Collision,Major Damage,NC,Single Vehicle Collision,priv-house-serv,Fire,CRV,...,4,4,2,3,-1.540445,-1.455287,-1.410987,-0.871839,-1.386294,0.362905


In [20]:
X_train1=X_train[['incident_severity_ordered','incident_state_ordered','insured_zip_new_ordered','insured_hobbies_ordered','insured_occupation_ordered','authorities_contacted_ordered','auto_model_ordered','vehicle_claim','collision_type_ordered','incident_type_ordered','umbrella_limit','capital-gains']]
X_test1=X_test[['incident_severity_ordered','incident_state_ordered','insured_zip_new_ordered','insured_hobbies_ordered','insured_occupation_ordered','authorities_contacted_ordered','auto_model_ordered','vehicle_claim','collision_type_ordered','incident_type_ordered','umbrella_limit','capital-gains']]

In [21]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(X_train1.values, i) for i in range(X_train1.shape[1])] ## Replace X with your indepedent data type
vif_s =pd.Series(vif, index =X_train1.columns)
print(vif_s)

incident_severity_ordered         3.136242
incident_state_ordered            4.277448
insured_zip_new_ordered          11.404597
insured_hobbies_ordered           2.923633
insured_occupation_ordered       10.422997
authorities_contacted_ordered     6.526248
auto_model_ordered                4.731074
vehicle_claim                    13.429581
collision_type_ordered           12.358026
incident_type_ordered            27.288596
umbrella_limit                    1.252461
capital-gains                     1.809924
dtype: float64


In [22]:
X_train1.head()

,incident_severity_ordered,incident_state_ordered,insured_zip_new_ordered,insured_hobbies_ordered,insured_occupation_ordered,authorities_contacted_ordered,auto_model_ordered,vehicle_claim,collision_type_ordered,incident_type_ordered,umbrella_limit,capital-gains
105,-2.239784,1,-0.847298,-2.538974,-1.142097,-2.772589,-1.029619,3360,1,1,0,34300
68,0.362905,1,-0.584513,-1.145132,-1.526056,-1.021651,-0.559616,49840,3,4,0,43300
479,0.362905,1,-1.145132,-0.847298,-1.049822,-0.871839,0.000000,66570,3,4,0,43700
399,-2.239784,3,-1.399366,-1.440362,-1.315677,-1.021651,-1.845827,57690,4,4,6000000,0
434,0.362905,2,-1.540445,-1.455287,-1.410987,-0.871839,-1.386294,46620,4,4,0,0


In [23]:
#Before calling the logistic regressor lets do the standard scaling
from sklearn.preprocessing import StandardScaler
X_train_m1=X_train1
X_test_m1=X_test1
y_train_m1=y_train
y_test_m1=y_test

# set up the scaler
scaler = StandardScaler()


In [24]:

# fit and transform train and test sets
X_train_scaled=scaler.fit_transform(X_train_m1)
X_test_scaled=scaler.fit_transform(X_test_m1)

# let's transform the returned NumPy arrays to dataframes 

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train1.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test1.columns)


In [25]:
import numpy as np
np.round(X_train1.describe(), 1)

,incident_severity_ordered,incident_state_ordered,insured_zip_new_ordered,insured_hobbies_ordered,insured_occupation_ordered,authorities_contacted_ordered,auto_model_ordered,vehicle_claim,collision_type_ordered,incident_type_ordered,umbrella_limit,capital-gains
count,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0
mean,-1.4,3.5,-1.2,-1.3,-1.2,-1.2,-1.2,38337.5,2.7,3.1,1141428.6,25268.9
std,1.2,1.9,0.3,0.9,0.4,0.6,0.6,18871.0,1.1,0.9,2309710.4,28117.9
min,-2.7,1.0,-1.8,-2.5,-2.1,-2.8,-3.0,70.0,1.0,1.0,-1000000.0,0.0
25%,-2.2,2.0,-1.5,-1.7,-1.4,-1.5,-1.6,31040.0,2.0,3.0,0.0,0.0
50%,-1.8,3.0,-1.1,-1.4,-1.1,-1.0,-1.1,42595.0,3.0,3.0,0.0,0.0
75%,0.4,6.0,-0.8,-1.0,-0.9,-0.9,-0.8,50822.5,4.0,4.0,0.0,51150.0
max,0.4,7.0,-0.6,1.4,-0.6,-0.7,0.1,79560.0,4.0,4.0,10000000.0,100500.0


In [26]:
np.round(X_train_scaled.describe(), 1)

,incident_severity_ordered,incident_state_ordered,insured_zip_new_ordered,insured_hobbies_ordered,insured_occupation_ordered,authorities_contacted_ordered,auto_model_ordered,vehicle_claim,collision_type_ordered,incident_type_ordered,umbrella_limit,capital-gains
count,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0,700.0
mean,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0
std,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
min,-1.1,-1.3,-1.8,-1.3,-2.6,-2.6,-2.8,-2.0,-1.5,-2.3,-0.9,-0.9
25%,-0.7,-0.8,-1.1,-0.4,-0.6,-0.5,-0.6,-0.4,-0.6,-0.1,-0.5,-0.9
50%,-0.3,-0.3,0.1,-0.2,0.1,0.3,0.2,0.2,0.3,-0.1,-0.5,-0.9
75%,1.6,1.3,0.9,0.3,0.7,0.6,0.6,0.7,1.3,0.9,-0.5,0.9
max,1.6,1.9,1.7,2.8,1.7,0.9,2.1,2.2,1.3,0.9,3.8,2.7


In [27]:
X_train1.head()

,incident_severity_ordered,incident_state_ordered,insured_zip_new_ordered,insured_hobbies_ordered,insured_occupation_ordered,authorities_contacted_ordered,auto_model_ordered,vehicle_claim,collision_type_ordered,incident_type_ordered,umbrella_limit,capital-gains
105,-2.239784,1,-0.847298,-2.538974,-1.142097,-2.772589,-1.029619,3360,1,1,0,34300
68,0.362905,1,-0.584513,-1.145132,-1.526056,-1.021651,-0.559616,49840,3,4,0,43300
479,0.362905,1,-1.145132,-0.847298,-1.049822,-0.871839,0.000000,66570,3,4,0,43700
399,-2.239784,3,-1.399366,-1.440362,-1.315677,-1.021651,-1.845827,57690,4,4,6000000,0
434,0.362905,2,-1.540445,-1.455287,-1.410987,-0.871839,-1.386294,46620,4,4,0,0


In [28]:
X_train_scaled.head()

,incident_severity_ordered,incident_state_ordered,insured_zip_new_ordered,insured_hobbies_ordered,insured_occupation_ordered,authorities_contacted_ordered,auto_model_ordered,vehicle_claim,collision_type_ordered,incident_type_ordered,umbrella_limit,capital-gains
0,-0.694541,-1.330748,0.930967,-1.336663,0.075931,-2.595745,0.322576,-1.854832,-1.544609,-2.322159,-0.494540,0.321418
1,1.567664,-1.330748,1.696222,0.146098,-1.009462,0.340479,1.048230,0.609966,0.319574,0.939737,-0.494540,0.641727
2,1.567664,-1.330748,0.063644,0.462933,0.336779,0.591706,1.912239,1.497145,0.319574,0.939737,-0.494540,0.655963
3,-0.694541,-0.267971,-0.676710,-0.167965,-0.414752,0.340479,-0.937593,1.026246,1.251666,0.939737,2.105046,-0.899316
4,1.567664,-0.799360,-1.087545,-0.183843,-0.684179,0.591706,-0.228106,0.439212,1.251666,0.939737,-0.494540,-0.899316


In [29]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()
classifier.fit(X_train_scaled,y_train)
y_pred=classifier.predict(X_test_scaled)
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score,f1_score,log_loss
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred))
print(recall_score(y_test,y_pred))
print(f1_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.8366666666666667
0.75
0.5454545454545454
0.631578947368421
[[209  14]
 [ 35  42]]


In [30]:
from sklearn.svm import SVC
model=SVC()
model.fit(X_train_scaled,y_train)
y_pred1=model.predict(X_test_scaled)
print(accuracy_score(y_test,y_pred1))
print(precision_score(y_test,y_pred1))
print(recall_score(y_test,y_pred1))
print(f1_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))

0.8433333333333334
0.7272727272727273
0.6233766233766234
0.6713286713286714
[[205  18]
 [ 29  48]]


In [31]:
from sklearn.neighbors import KNeighborsClassifier
model1=KNeighborsClassifier()
model1.fit(X_train_scaled,y_train)
y_pred2=model1.predict(X_test_scaled)
print(confusion_matrix(y_test,y_pred2))
print(precision_score(y_test,y_pred2))
print(recall_score(y_test,y_pred2))
print(accuracy_score(y_test,y_pred2))
print(f1_score(y_test,y_pred2))

[[205  18]
 [ 37  40]]
0.6896551724137931
0.5194805194805194
0.8166666666666667
0.5925925925925926


In [32]:
from sklearn.naive_bayes import GaussianNB
model2=GaussianNB()
model2.fit(X_train_scaled,y_train)
y_pred3=model2.predict(X_test_scaled)
print(confusion_matrix(y_test,y_pred3))
print(precision_score(y_test,y_pred3))
print(recall_score(y_test,y_pred3))
print(accuracy_score(y_test,y_pred3))
print(f1_score(y_test,y_pred3))

[[194  29]
 [ 21  56]]
0.6588235294117647
0.7272727272727273
0.8333333333333334
0.691358024691358


In [33]:
from sklearn.ensemble import RandomForestClassifier
model3=RandomForestClassifier()
model3.fit(X_train1,y_train)
y_pred4=model3.predict(X_test1)
print(confusion_matrix(y_test,y_pred4))
print(precision_score(y_test,y_pred4))
print(recall_score(y_test,y_pred4))
print(accuracy_score(y_test,y_pred4))
print(f1_score(y_test,y_pred4))

[[209  14]
 [ 34  43]]
0.7543859649122807
0.5584415584415584
0.84
0.6417910447761195


In [34]:
#lets try to feed this into pycaret library
X.head()

,vehicle_claim,umbrella_limit,capital-gains,collision_type,incident_severity,incident_state,incident_type,insured_occupation,authorities_contacted,auto_model,insured_hobbies,insured_zip_new,target
0,52080,0,53300,Side Collision,Major Damage,SC,Single Vehicle Collision,craft-repair,Police,92x,sleeping,46,1
1,3510,5000000,0,Missing,Minor Damage,VA,Vehicle Theft,machine-op-inspct,Police,E400,reading,46,1
2,23100,5000000,35100,Rear Collision,Minor Damage,NY,Multi-vehicle Collision,sales,Police,RAM,board-games,43,0
3,50720,6000000,48900,Front Collision,Major Damage,OH,Single Vehicle Collision,armed-forces,Police,Tahoe,board-games,60,1
4,4550,6000000,66000,Missing,Minor Damage,NY,Vehicle Theft,sales,None,RSX,board-games,61,0


In [330]:
#Write the final output to an excel file
writer = pd.ExcelWriter(r'C:\Users\ADMIN\Desktop\UseCase2_PycaretInput.xlsx')
X.to_excel(writer,index = False, header=True)
writer.save()

In [35]:
from sklearn.ensemble import AdaBoostClassifier
model4=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                    n_estimators=50, random_state=5471)
model4.fit(X_train1,y_train)
y_pred5=model4.predict(X_test1)
print(confusion_matrix(y_test,y_pred5))
print(precision_score(y_test,y_pred5))
print(recall_score(y_test,y_pred5))
print(accuracy_score(y_test,y_pred5))
print(f1_score(y_test,y_pred5))

[[208  15]
 [ 43  34]]
0.6938775510204082
0.44155844155844154
0.8066666666666666
0.5396825396825397


Over sampling approach

In [36]:
from imblearn.over_sampling import SMOTE


In [37]:
X.shape,y.shape

((1000, 13), (1000,))

In [38]:
#Trying to use monotonicity encoding for collision_type and incident_state
CategoricalImputation_WOE(X,X,'collision_type','target')
CategoricalImputation_WOE(X,X,'incident_type','target')
CategoricalImputation_WOE(X,X,'incident_state','target')


In [39]:
#Use WOE encoding for other categorical columns
CategoricalImputation_WOE(X,X,'insured_zip_new','target')
CategoricalImputation_WOE(X,X,'insured_hobbies','target')
CategoricalImputation_WOE(X,X,'insured_occupation','target')
CategoricalImputation_WOE(X,X,'authorities_contacted','target')
CategoricalImputation_WOE(X,X,'auto_model','target')
CategoricalImputation_WOE(X,X,'incident_severity','target')

In [40]:
#Use ordinal encoding for incident_severity
incident_severity_map = {'Trivial Damage':1,
               'Minor Damage':2,
               'Major Damage':3,
               'Total Loss':4,
               
}

X_train['incident_severity_number'] = X_train.incident_severity.map(incident_severity_map)
X_test['incident_severity_number'] = X_test.incident_severity.map(incident_severity_map)
X_train.head()

,vehicle_claim,umbrella_limit,capital-gains,collision_type,incident_severity,incident_state,incident_type,insured_occupation,authorities_contacted,auto_model,...,collision_type_ordered,incident_type_ordered,incident_state_ordered,incident_severity_number,insured_zip_new_ordered,insured_hobbies_ordered,insured_occupation_ordered,authorities_contacted_ordered,auto_model_ordered,incident_severity_ordered
105,3360,0,34300,Missing,Minor Damage,WV,Vehicle Theft,prof-specialty,None,E400,...,1,1,1,2,-0.847298,-2.538974,-1.142097,-2.772589,-1.029619,-2.239784
68,49840,0,43300,Front Collision,Major Damage,WV,Single Vehicle Collision,machine-op-inspct,Ambulance,A5,...,3,4,1,3,-0.584513,-1.145132,-1.526056,-1.021651,-0.559616,0.362905
479,66570,0,43700,Front Collision,Major Damage,WV,Single Vehicle Collision,sales,Fire,ML350,...,3,4,1,3,-1.145132,-0.847298,-1.049822,-0.871839,0.000000,0.362905
399,57690,6000000,0,Rear Collision,Minor Damage,NY,Single Vehicle Collision,armed-forces,Ambulance,Malibu,...,4,4,3,2,-1.399366,-1.440362,-1.315677,-1.021651,-1.845827,-2.239784
434,46620,0,0,Rear Collision,Major Damage,NC,Single Vehicle Collision,priv-house-serv,Fire,CRV,...,4,4,2,3,-1.540445,-1.455287,-1.410987,-0.871839,-1.386294,0.362905


In [41]:
X=X[['incident_severity_ordered','incident_state_ordered','insured_zip_new_ordered','insured_hobbies_ordered','insured_occupation_ordered','authorities_contacted_ordered','auto_model_ordered','vehicle_claim','collision_type_ordered','incident_type_ordered','umbrella_limit','capital-gains']]

In [42]:
X.shape,y.shape

((1000, 12), (1000,))

In [43]:

x_resample, y_resample  = SMOTE().fit_sample(X, y.values.ravel())

print(x_resample.shape)
print(y_resample.shape)

(1506, 12)
(1506,)


In [59]:
x_train2, x_test2, y_train2, y_test2 = train_test_split(x_resample, y_resample, test_size = 0.2, random_state = 0)

print(x_train2.shape)
print(y_train2.shape)
print(x_test2.shape)
print(y_test2.shape)

(1204, 12)
(1204,)
(302, 12)
(302,)


In [45]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train2 = sc.fit_transform(x_train2)
x_test2 = sc.transform(x_test2)

In [51]:
from sklearn.metrics import classification_report

In [60]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

model_o = RandomForestClassifier()
model_o.fit(x_train2, y_train2)

y_pred = model_o.predict(x_test2)

print("Training Accuracy: ", model_o.score(x_train2, y_train2))
print('Testing Accuarcy: ', model_o.score(x_test2, y_test2))

# classification report
cr = classification_report(y_test2, y_pred)
print(cr)

Training Accuracy:  1.0
Testing Accuarcy:  0.8708609271523179
              precision    recall  f1-score   support

           0       0.87      0.88      0.87       154
           1       0.87      0.86      0.87       148

    accuracy                           0.87       302
   macro avg       0.87      0.87      0.87       302
weighted avg       0.87      0.87      0.87       302



In [57]:
print(confusion_matrix(y_test2,y_pred))
print(precision_score(y_test2,y_pred))
print(recall_score(y_test2,y_pred))
print(accuracy_score(y_test2,y_pred))
print(f1_score(y_test2,y_pred))

[[136  18]
 [ 19 129]]
0.8775510204081632
0.8716216216216216
0.8774834437086093
0.8745762711864407


In [54]:
from sklearn.naive_bayes import GaussianNB
model2=GaussianNB()
model2.fit(x_train2,y_train2)
y_pred3=model2.predict(x_test2)
print(confusion_matrix(y_test2,y_pred3))
print(precision_score(y_test2,y_pred3))
print(recall_score(y_test2,y_pred3))
print(accuracy_score(y_test2,y_pred3))
print(f1_score(y_test2,y_pred3))

[[ 80  74]
 [ 15 133]]
0.642512077294686
0.8986486486486487
0.7052980132450332
0.7492957746478872
